In [1]:
!pip install -r "/requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
!pip install bitsandbytes

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `model_test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `model_test`


In [4]:
import torch
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score
from peft import LoraConfig, get_peft_model
import evaluate
from tqdm import tqdm
from ast import literal_eval
from transformers import BitsAndBytesConfig
from datasets import Dataset

In [5]:
# 테스트 데이터 로드
test_data = pd.read_csv('/test.csv')

# 데이터 처리
records = []
for _, row in test_data.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        "question_plus": problems.get('question_plus', None),
    }
    records.append(record)

df = pd.DataFrame(records)
test_dataset = Dataset.from_pandas(df)

In [6]:
# 8-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8-bit 양자화로 변경
    bnb_8bit_compute_dtype=torch.float16,  # 8-bit 양자화에서 연산 시 데이터 타입
)

# 학습된 모델 디렉토리
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

# 모델 로드
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,  # 양자화 설정 적용
#     device_map="auto",              # GPU로 자동 배치
#     trust_remote_code=True
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,  # 16-bit 사용
    trust_remote_code=True
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# LoRA 설정 (학습 시 사용한 설정과 동일해야 함)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.eval()  # 추론 모드로 전환

Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ExaoneForCausalLM(
      (transformer): ExaoneModel(
        (wte): Embedding(102400, 4096, padding_idx=0)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-31): 32 x ExaoneBlock(
            (ln_1): ExaoneRMSNorm()
            (attn): ExaoneAttention(
              (attention): ExaoneSdpaAttention(
                (rotary): ExaoneRotaryEmbedding()
                (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
             

In [7]:
# 프롬프트 생성 함수
PROMPT_NO_QUESTION_PLUS = """지문:
{paragraph}

질문:
{question}

선택지:
{choices}

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

PROMPT_QUESTION_PLUS = """지문:
{paragraph}

질문:
{question}

<보기>:
{question_plus}

선택지:
{choices}

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

In [8]:
def create_prompt(example):
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(example["choices"])])
    if example["question_plus"]:
        prompt = PROMPT_QUESTION_PLUS.format(
            paragraph=example["paragraph"],
            question=example["question"],
            question_plus=example["question_plus"],
            choices=choices_string,
        )
    else:
        prompt = PROMPT_NO_QUESTION_PLUS.format(
            paragraph=example["paragraph"],
            question=example["question"],
            choices=choices_string,
        )
    return prompt

In [9]:
# 예측 수행
infer_results = []
pred_choices_map = {0: "1", 1: "2", 2: "3", 3: "4", 4: "5"}

model.eval()
with torch.inference_mode():
    for data in tqdm(test_dataset):
        _id = data["id"]
        prompt = create_prompt(data)

        # 입력 텍스트를 토큰화
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        ).to("cuda")

        # token_type_ids 제거
        inputs.pop("token_type_ids", None)

        # 모델 예측 수행
        outputs = model(**inputs)
        logits = outputs.logits[:, -1]  # 마지막 토큰의 로짓만 사용

        # 선택지에 해당하는 로짓 추출
        target_logit_list = [
            logits[0, tokenizer.convert_tokens_to_ids(str(i + 1))].item() for i in range(5)
        ]

        # 소프트맥스 계산
        probs = torch.nn.functional.softmax(torch.tensor(target_logit_list), dim=-1).cpu().numpy()

        # 가장 높은 확률의 선택지 예측
        predict_value = pred_choices_map[np.argmax(probs)]
        infer_results.append({"id": _id, "answer": predict_value})

# 결과 저장
result_df = pd.DataFrame(infer_results)
result_df.to_csv("/predictions.csv", index=False)

100%|██████████| 869/869 [04:26<00:00,  3.26it/s]
